## Lectura 4-3: Supuestos de la regresión lineal simple

### Interpretación probabilística

- Teniendo en cuenta los supuestos anteriores, estamos diciendo que la densidad de probabilidad (PDF) de los errores $\epsilon$ es una gaussiana con media cero y varianza constante $\sigma^2$:

 $$\text{PDF}(\epsilon_{i})=\frac{1}{\sqrt{2\pi} \sigma} \exp \left(- \frac{\epsilon_{i}^{2}}{2\sigma^2}\right)$$

-   Además todos los $\epsilon_{i}$ son IID (independientes e idénticamente distribuidos).

- Esto implica que:

 $$\text{PDF}(y_i|x_{i};\beta)=\frac{1}{\sqrt{2\pi} \sigma} \exp \left(- \frac{(y_i - h_{\beta}(x_{i}) )^{2}}{2\sigma^2}\right)$$

-  Lo que implica que la distribución de $\mathbf{y}$ dados los valores de $\mathbf{x}$, parametrizados por $\beta$ también se distribuye normalmente.

- Estimemos los parámetros del modelo ($\beta$) utilizando la estimación de máxima verosimilitud.


-   La función de verosimilitud de $\beta$ puede escribirse como:

$$\mathcal{L}(\beta) = \prod_{i=1}^{n}\frac{1}{\sqrt{2\pi} \sigma} \exp \left(- \frac{(y_i - h_{\beta}(x_{i}) )^{2}}{2\sigma^2}\right)$$

-   y el logaritmo de probabilidad $l_n(\beta)$:

$$l_n(\beta)  =  \log  \mathcal{L}(\beta) $$
$$ = \log \prod_{i=1}^{n}\frac{1}{\sqrt{2\pi} \sigma} \exp \left(- \frac{(y_i - h_{\beta}(x_{i}) )^{2}}{2\sigma^2}\right) $$
$$ = \sum_{i=1}^n  \log \frac{1}{\sqrt{2\pi} \sigma} \exp \left(- \frac{(y_i - h_{\beta}(x_{i}) )^{2}}{2\sigma^2}\right) $$
$$ = n \log\left(\frac{1}{\sqrt{2\pi} \sigma}\right) - \frac{1}{\sigma^2}\times \frac{1}{2}\sum_{i=1}^n(y_i - h_{\beta}(x_{i}) )^{2}$$

-   Después de ignorar las constantes, nos damos cuenta de que maximizar $l_n(\beta)$ es equivalente a minimizar:

$$\sum_{i=1}^n(y_i - h_{\beta}(x_{i}))^{2}$$

-   Lo que equivale a minimizar la SSE.

- Entonces, si uno estima los parámetros de $\beta$ utilizando la estimación de máxima verosimilitud se llega a los mismos resultados que haciendo la estimación por mínimos cuadrados.

- Por lo tanto, cada vez que realizamos una estimación por mínimos cuadrados, implícitamente estamos haciendo los mismos supuestos probabilísticos mencionados anteriormente (por ejemplo, normalidad, homocedasticidad).


### Ejemplo: un modelo de altura

- Vamos a trabajar con el conjunto de datos `Howell1` que contiene datos demográficos de los !Kung San del Kalahari recogidos por Nancy Howell a finales de los años sesenta.

- Los !Kung San son la población forrajera más famosa del siglo XX, en gran parte gracias a los detallados estudios cuantitativos realizados por personas como Howell.


-   Cada observación corresponde a un individuo.

- Las variables del conjunto de datos son:

- Exploremos las correlaciones lineales entre las variables

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [2]:
data = '/Users/santiagoromero/Documents/Maestria/Modelos Analisis Estadisticos/Ejercicios/howell1.csv'
data_frame = pd.read_csv(data, sep=',')
data_frame.head()

,height,weight,age,male
0,151.765,47.825606,63.0,1
1,139.700,36.485807,63.0,0
2,136.525,31.864838,65.0,0
3,156.845,53.041914,41.0,1
4,145.415,41.276872,51.0,0


In [3]:
# Escriba su código aquí...
data_frame.corr()

,height,weight,age,male
height,1.000000,0.940822,0.683689,0.139229
weight,0.940822,1.000000,0.678335,0.155443
age,0.683689,0.678335,1.000000,0.005887
male,0.139229,0.155443,0.005887,1.000000


- Podemos ver que existe una correlación positiva entre `altura`  y `edad`.
- Filtremos los ejemplos no adultos porque sabemos que la altura está fuertemente correlacionada con la edad antes de la edad adulta.


In [4]:
# Escriba su código aquí...
df_menores = data_frame[data_frame['age']<18]
df_menores.head()

,height,weight,age,male
18,121.92,19.617854,12.0,1
19,105.41,13.947954,8.0,0
20,86.36,10.489315,6.5,0
23,129.54,23.586784,13.0,1
24,109.22,15.989118,7.0,0


-   Ahora bien, la edad no se correlaciona con la altura:

In [5]:
## Escriba su código aquí...
df_menores.corr()

,height,weight,age,male
height,1.000000,0.944479,0.943313,0.061724
weight,0.944479,1.000000,0.917491,0.071611
age,0.943313,0.917491,1.000000,-0.006452
male,0.061724,0.071611,-0.006452,1.000000


In [6]:
df_mayores = data_frame[data_frame['age']>17]
df_mayores.head()

,height,weight,age,male
0,151.765,47.825606,63.0,1
1,139.700,36.485807,63.0,0
2,136.525,31.864838,65.0,0
3,156.845,53.041914,41.0,1
4,145.415,41.276872,51.0,0


In [7]:
df_mayores.corr()

,height,weight,age,male
height,1.000000,0.754748,-0.101838,0.699993
weight,0.754748,1.000000,-0.172904,0.524453
age,-0.101838,-0.172904,1.000000,0.028455
male,0.699993,0.524453,0.028455,1.000000


-  Modelemos la altura en función del peso mediante una regresión lineal simple:
    
$$\text{height}(\text{weight})=\beta_0+\beta_1*\text{weight}$$


In [14]:
# Escriba su código aquí...
import statsmodels.formula.api as smf

# estimate the model and assign the result to linear_model
linear_model = smf.ols(formula='height ~ weight', data=df_mayores).fit()



- Podemos ver que los coeficientes del modelo son $\beta_{0}=113,879$ y $\beta_{1}=0,905$.

- La estimación de $\beta_{0}=xxxx$, indica que una persona de peso 0 debería medir alrededor de $114$cm, lo cual sabemos ....

- Dado que $\beta_1$ es una pendiente, el valor $xxx$ puede leerse como que se espera que una persona con 1 kg más de peso sea xxx cm más alta.

- Podemos acceder directamente a los coeficientes y almacenarlos en una variable:

In [ ]:
# Escriba su código aquí...

-   Podemos ver varios indicadores sobre el modelo lineal con el comando **summary**:

In [13]:
# Escriba su código aquí...
print(linear_model.summary())

                            OLS Regression Results                            
Dep. Variable:                 height   R-squared:                       0.570
Model:                            OLS   Adj. R-squared:                  0.568
Method:                 Least Squares   F-statistic:                     463.3
Date:                Fri, 08 Sep 2023   Prob (F-statistic):           4.68e-66
Time:                        21:24:15   Log-Likelihood:                -1071.0
No. Observations:                 352   AIC:                             2146.
Df Residuals:                     350   BIC:                             2154.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    113.8794      1.911     59.589      0.0

- Podemos ver que $\beta_0$ y $\beta_1$ son ambos estadísticamente significativamente diferentes de cero.

- Vemos que el coeficiente de determinación $R^2$ tiene un valor de 0,57$ que no es muy bueno pero aceptable.

- Podemos concluir que el peso, aunque ....

- Podemos acceder al coeficiente de determinación:

In [ ]:
# Escriba su código aquí...

- También podemos acceder a los valores ajustados que son los valores predichos por mi modelo para los datos utilizados:

In [ ]:
# Escriba su código aquí...

- Podemos comprobar ahora que todas las definiciones dadas para $R^2$ son
    equivalentes.

In [ ]:
# Escriba su código aquí...

-   O calcular manualmente el error estándar del modelo $SE_{\text{model}}$:

In [ ]:
# Escriba su código aquí...



-  Supongamos ahora que conocemos el peso de dos personas !Kung pero no conocemos su altura.

- Podríamos utilizar nuestro modelo lineal para predecir la altura de estas dos personas.



In [ ]:
# Escriba su código aquí...